Goal:
classify a task for a given student as (1) too easy, (2) just right, or (3) too difficult.

Data:
- Robotanist, order and times of solved tasks
- proxy for diffulty levels:
- too easy = less than 1 minute
- too difficult = more than 20 minutes

Ideally, the same methods should be usable in scenario where we have explicit user information (qualitative data) about perceived difficulty (obtained via the flow-level question).

Usage:
- soft recommendation via color each task (too easy, just right, too difficult)
- soft recommendation via coarse time predictions (1m, 10m, >20m)
- set challenge level (1m or 10m)
- hard recommendation: hide/lock too easy/difficult tasks

In [60]:
%matplotlib inline
from collections import OrderedDict
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Data

In [114]:
ordering = pd.read_csv('../data/robotanik/user_time_ordering.csv').set_index('Login')
# Make task IDs integers (from string labels).
ordering.columns = map(int, ordering.columns)
ordering.head()

,635,636,637,638,639,640,641,642,643,644,...,1402,1403,1404,1405,1406,1407,1704,1705,1706,1707
Login,,,,,,,,,,,,,,,,,,,,,
U1,1.0,2.0,13.0,5.0,NaN,9.0,NaN,NaN,NaN,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
U2,1.0,2.0,4.0,3.0,5.0,6.0,9.0,10.0,8.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,58.0,59.0,60.0,61.0
U4,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
U5,1.0,2.0,3.0,4.0,16.0,5.0,15.0,NaN,11.0,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
U6,1.0,2.0,3.0,7.0,NaN,8.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
times = pd.read_csv('../data/robotanik/user_time.csv').set_index('Login')
# Make task IDs integers (from string labels).
times.columns = map(int, times.columns)
# We will work with log times; TODO: explain why
times = times.applymap(np.log)
times.head()

,635,636,637,638,639,640,641,642,643,644,...,1402,1403,1404,1405,1406,1407,1704,1705,1706,1707
Login,,,,,,,,,,,,,,,,,,,,,
U1,3.218876,2.564949,2.639057,3.637586,NaN,6.129050,NaN,NaN,NaN,4.882802,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
U2,2.302585,1.609438,3.891820,2.944439,4.521789,3.135494,4.043051,3.688879,3.737670,5.351858,...,NaN,NaN,NaN,NaN,NaN,NaN,5.220356,4.727388,3.583519,4.290459
U4,3.258097,1.791759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
U5,3.951244,3.663562,3.218876,2.944439,5.897154,3.218876,4.595120,NaN,4.025352,3.828641,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
U6,4.219508,2.995732,3.465736,2.890372,NaN,3.737670,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The dataset contains about 10K of users and 80 tasks.

In [116]:
times.shape

(10605, 78)

TODO: split data randomly into train/validation/test sets

In [117]:
train_times = times.head(10)
train_order = ordering.head(10)

# Explorative Analysis

# Features

Start with 2 features:
1. how many tasks has the student already solved (student_solved_percentage)
2. how many students have solved the task (task_solved_percentage)

Feature ideas:
- student-related (skill):
    - number of solved tasks
    - number of solved tasks in easy/flow/difficult mode
    - the most difficult solved task [in easy/flow/difficult mode]
    - which tasks solved [in which mode] / log-times / quartiles
- task-related (difficulty)
    - how many students have solved the task (percentage)
    - mean/median order
    - median log-time
    - percentage of easy/flow/difficult attempts
- content based

In [118]:
train_order

,635,636,637,638,639,640,641,642,643,644,...,1402,1403,1404,1405,1406,1407,1704,1705,1706,1707
Login,,,,,,,,,,,,,,,,,,,,,
U1,1.0,2.0,13.0,5.0,NaN,9.0,NaN,NaN,NaN,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
U2,1.0,2.0,4.0,3.0,5.0,6.0,9.0,10.0,8.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,58.0,59.0,60.0,61.0
U4,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
U5,1.0,2.0,3.0,4.0,16.0,5.0,15.0,NaN,11.0,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
U6,1.0,2.0,3.0,7.0,NaN,8.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
U7,1.0,2.0,NaN,NaN,4.0,NaN,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
U8,1.0,2.0,5.0,6.0,16.0,4.0,23.0,NaN,11.0,12.0,...,NaN,35.0,NaN,NaN,NaN,NaN,31.0,33.0,34.0,32.0
U9,1.0,2.0,5.0,3.0,13.0,4.0,7.0,9.0,8.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
U10,1.0,2.0,4.0,5.0,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [119]:
task_features = pd.DataFrame({
  'time_median': train_times.median(),
  'solved_percentage': train_times.count() / len(train_times)
})
task_features.head()

,solved_percentage,time_median
635,1.0,3.329647
636,1.0,2.381087
637,0.8,3.310037
638,0.8,3.138322
639,0.6,4.516324


In [120]:
# create a dataframe of events (student-task interactions)
task_count = train_order.shape[1]
data = []
for user_id in train_order.index:
    order = train_order.loc[user_id].dropna().sort_values()
    for task_id, student_order in order.items():
        event = OrderedDict(
            student_id=int(user_id[2:]),
            student_order=int(student_order),
            student_solved_percentage=(student_order-1)/task_count,
            task_id=int(task_id),
            time=train_times.loc[user_id, task_id])
        data.append(event)


events = pd.DataFrame(data)
events.head()

,student_id,student_order,student_solved_percentage,task_id,time
0,1,1,0.000000,635,3.218876
1,1,2,0.012821,636,2.564949
2,1,3,0.025641,656,4.595120
3,1,4,0.038462,698,3.850148
4,1,5,0.051282,638,3.637586


In [122]:
# join events with task features
# TODO: reset index
# TODO: sort by (student_id, student_order)
events = pd.merge(events, task_features, left_on='task_id', right_index=True)
events.head()

,student_id,student_order,student_solved_percentage,task_id,time,solved_percentage,time_median
0,1,1,0.0,635,3.218876,1.0,3.329647
14,2,1,0.0,635,2.302585,1.0,3.329647
75,4,1,0.0,635,3.258097,1.0,3.329647
78,5,1,0.0,635,3.951244,1.0,3.329647
101,6,1,0.0,635,4.219508,1.0,3.329647
